<div style="font-size:18pt; padding-top:20px; text-align:center">ЗАДАНИЕ 3. <b>Рекомендательные системы и </b> <span style="font-weight:bold; color:green">Spark MLlib</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru, papulin.ru_hse@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Варианты</a></li>
        <li><a href="#2">Задание</a></li>
    </ol>
</div>

<p>Подключение стилей оформления</p>

In [1]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

<div class="msg-block msg-imp">
  <div class="msg-text-imp">
      <p>СРОК СДАЧИ: <b>-</b></p>
  </div>
</div>

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Варианты</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Чтобы узнать свой вариант, введите Вашу фамилию в соответвующее поле ниже и запустите ячейку:</p>

In [2]:
surname = "Кирьяненко" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  2
Задача № 2:  2


<div class="msg-block msg-warning">
  <p class="msg-text-warn">Варианты могут не совпадать с прошлыми ДЗ</p>
</div>

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Задание</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [3]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, LongType, DateType, TimestampType
from pyspark.sql.functions import desc

conf = pyspark.SparkConf() \
        .setAppName("mllibRDDApp") \
        .set("spark.submit.deployMode", "client")
sc = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(sc)

print('Start Spark app')

Start Spark app


<p><b>Датасеты:</b></p>

Небольшой набор данных: <a href="http://files.grouplens.org/datasets/movielens/ml-latest-small.zip">http://files.grouplens.org/datasets/movielens/ml-latest-small.zip</a><br>
Полный набор: <a href="http://files.grouplens.org/datasets/movielens/ml-latest.zip">http://files.grouplens.org/datasets/movielens/ml-latest.zip</a>

<div class="msg-block msg-warning">
  <div class="msg-text-warn">
      <p style="margin:0px">1. Сначала выполните указанные ниже задачи для небольшого набора данных, потом для полного</p>
      <p style="margin:0px">2. Использовать Dataframe. Исключением может быть задача 2</p>
  </div>
</div>

<p><b>Задача 1.</b> Анализ датасета <b><i>(2 балла)</i></b></p>

<p><b style="color:green;">Вариант 1.</b> Animation, Romance, Documentary</p>
<p><b style="color:green;">Вариант 2.</b> Drama, Comedy, Musical</p>
<p><b style="color:green;">Вариант 3.</b> Thriller, Sci-Fi, Adventure</p>

<div class="msg-block msg-info">
  <p class="msg-text-info">Один фильм может принадлежать разным жанрам</p>
</div>

<p>1. Выведите данные, сопоставляющие жанры и количество фильмов</p>

In [4]:
# Читаю файл с фильмами
movies_file = 'data/ml-latest-small/movies.csv'
movies_schema = StructType([StructField(name="id", dataType=LongType(), nullable=False),
                            StructField(name="title", dataType=StringType(), nullable=False),
                            StructField(name="genres", dataType=StringType(), nullable=False)])
movies = sqlContext.read.load(path=movies_file, format="csv", schema=movies_schema, header="true",
                             inferSchema="false", sep=",", nullValue="null", mode="DROPMALFORMED")
movies.show()

# Создаю связывающую таблицу фильм - жанр
movies_genres = movies.rdd \
    .flatMap(lambda row: [(row.id, genre) for genre in row.genres.split('|')]) \
    .toDF(['movie_id', 'genre'])
movies_genres.show()

# Сопоставляю жанры и количество фильмов
genre_movie_cnt = movies_genres.groupBy('genre').count()
print('Жанры и количество фильмов:')
genre_movie_cnt.show()

+---+--------------------+--------------------+
| id|               title|              genres|
+---+--------------------+--------------------+
|  1|    Toy Story (1995)|Adventure|Animati...|
|  2|      Jumanji (1995)|Adventure|Childre...|
|  3|Grumpier Old Men ...|      Comedy|Romance|
|  4|Waiting to Exhale...|Comedy|Drama|Romance|
|  5|Father of the Bri...|              Comedy|
|  6|         Heat (1995)|Action|Crime|Thri...|
|  7|      Sabrina (1995)|      Comedy|Romance|
|  8| Tom and Huck (1995)|  Adventure|Children|
|  9| Sudden Death (1995)|              Action|
| 10|    GoldenEye (1995)|Action|Adventure|...|
| 11|American Presiden...|Comedy|Drama|Romance|
| 12|Dracula: Dead and...|       Comedy|Horror|
| 13|        Balto (1995)|Adventure|Animati...|
| 14|        Nixon (1995)|               Drama|
| 15|Cutthroat Island ...|Action|Adventure|...|
| 16|       Casino (1995)|         Crime|Drama|
| 17|Sense and Sensibi...|       Drama|Romance|
| 18|   Four Rooms (1995)|              

<p>2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соотвествии с вариантом</p>

In [5]:
# Читаю файл с отзывами
ratings_file = 'data/ml-latest-small/ratings.csv'
vote_schema = StructType([StructField(name="user_id", dataType=LongType(), nullable=False),
                          StructField(name="movie_id", dataType=LongType(), nullable=False),
                          StructField(name="vote", dataType=DoubleType(), nullable=False),
                          StructField(name="date", dataType=LongType(), nullable=False)])
votes = sqlContext.read.load(path=ratings_file, format="csv", schema=vote_schema, header="true",
                             inferSchema="false", sep=",", nullValue="null", mode="DROPMALFORMED")
votes.show()

# Подсчитываю количество голосов для каждого фильма
movies_votes_cnt = movies \
    .join(votes, movies.id == votes.movie_id, how='left') \
    .groupBy('id', 'title') \
    .agg({'*': 'count'}) \
    .withColumnRenamed('count(1)', 'votes_cnt')
movies_votes_cnt.show()

# Для каждого жанра вывожу 10 популярных фильмов
VARIANT_GENRES = ['Drama', 'Comedy', 'Musical']
for genre in VARIANT_GENRES:
    print('10 популярных фильмов жанра "{}":'.format(genre))
    movies_votes_cnt \
        .join(movies_genres, movies.id == movies_genres.movie_id) \
        .filter(movies_genres.genre == genre) \
        .orderBy(desc('votes_cnt')) \
        .limit(10) \
        .show()

+-------+--------+----+---------+
|user_id|movie_id|vote|     date|
+-------+--------+----+---------+
|      1|       1| 4.0|964982703|
|      1|       3| 4.0|964981247|
|      1|       6| 4.0|964982224|
|      1|      47| 5.0|964983815|
|      1|      50| 5.0|964982931|
|      1|      70| 3.0|964982400|
|      1|     101| 5.0|964980868|
|      1|     110| 4.0|964982176|
|      1|     151| 5.0|964984041|
|      1|     157| 5.0|964984100|
|      1|     163| 5.0|964983650|
|      1|     216| 5.0|964981208|
|      1|     223| 3.0|964980985|
|      1|     231| 5.0|964981179|
|      1|     235| 4.0|964980908|
|      1|     260| 5.0|964981680|
|      1|     296| 3.0|964982967|
|      1|     316| 3.0|964982310|
|      1|     333| 5.0|964981179|
|      1|     349| 4.0|964982563|
+-------+--------+----+---------+
only showing top 20 rows

+-----+--------------------+---------+
|   id|               title|votes_cnt|
+-----+--------------------+---------+
|   55|      Georgia (1995)|        1|
| 

<p>3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соотвествии с вариантом</p>

In [6]:
for genre in VARIANT_GENRES:
    print('10 не популярных фильмов жанра "{}":'.format(genre))
    movies_votes_cnt \
        .join(movies_genres, movies.id == movies_genres.movie_id) \
        .filter(movies_genres.genre == genre) \
        .filter('votes_cnt > 10') \
        .orderBy('votes_cnt') \
        .limit(10) \
        .show()

10 не популярных фильмов жанра "Drama":
+------+--------------------+---------+--------+-----+
|    id|               title|votes_cnt|movie_id|genre|
+------+--------------------+---------+--------+-----+
|  3824|Autumn in New Yor...|       11|    3824|Drama|
|  5108|       John Q (2002)|       11|    5108|Drama|
|  1041|Secrets & Lies (1...|       11|    1041|Drama|
|  3741|     Badlands (1973)|       11|    3741|Drama|
|  5693|Saturday Night Fe...|       11|    5693|Drama|
|137857|The Jungle Book (...|       11|  137857|Drama|
| 48082|Science of Sleep,...|       11|   48082|Drama|
|  1480|Smilla's Sense of...|       11|    1480|Drama|
|  1483|        Crash (1996)|       11|    1483|Drama|
|   965|39 Steps, The (1935)|       11|     965|Drama|
+------+--------------------+---------+--------+-----+

10 не популярных фильмов жанра "Comedy":
+------+--------------------+---------+--------+------+
|    id|               title|votes_cnt|movie_id| genre|
+------+--------------------+-------

<p>4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соотвествии с вариантом</p>

In [7]:
# Подсчитываю средний рейтинг и количество голосов для каждого фильма
movies_ratings = movies \
    .join(votes, movies.id == votes.movie_id, how='left') \
    .groupBy('id', 'title') \
    .agg({'*': 'count', 'vote': 'avg'}) \
    .withColumnRenamed('count(1)', 'votes_cnt') \
    .withColumnRenamed('avg(vote)', 'rating')
movies_ratings.show()

for genre in VARIANT_GENRES:
    print('Топ 10 фильмов жанра "{}":'.format(genre))
    movies_ratings \
        .join(movies_genres, movies.id == movies_genres.movie_id) \
        .filter(movies_genres.genre == genre) \
        .filter('votes_cnt > 10') \
        .orderBy(desc('rating')) \
        .limit(10) \
        .show()

+-----+--------------------+---------+------------------+
|   id|               title|votes_cnt|            rating|
+-----+--------------------+---------+------------------+
|   55|      Georgia (1995)|        1|               4.0|
|  502|Next Karate Kid, ...|       15|2.3666666666666667|
| 1301|Forbidden Planet ...|       13|3.8076923076923075|
| 1458|        Touch (1997)|        1|               4.0|
| 1526| Fathers' Day (1997)|        3|               1.0|
| 2887|    Simon Sez (1999)|        1|               1.0|
| 3070|Adventures of Buc...|       21|3.5476190476190474|
| 3129|Sweet and Lowdown...|        3|3.6666666666666665|
| 3448|Good Morning, Vie...|       43| 3.802325581395349|
| 3702|      Mad Max (1979)|       38| 3.486842105263158|
| 3965|Strange Love of M...|        1|               3.5|
| 4285|Frankie and Johnn...|        3|2.8333333333333335|
| 5221|Harrison's Flower...|        1|               3.0|
| 5250|   Stir Crazy (1980)|        3|               4.0|
| 6193|Poolhal

<p>5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соотвествии с вариантом</p>

In [8]:
for genre in VARIANT_GENRES:
    print('Топ 10 худших фильмов жанра "{}":'.format(genre))
    movies_ratings \
        .join(movies_genres, movies.id == movies_genres.movie_id) \
        .filter(movies_genres.genre == genre) \
        .filter('votes_cnt > 10') \
        .orderBy('rating') \
        .limit(10) \
        .show()

Топ 10 худших фильмов жанра "Drama":
+-----+--------------------+---------+------------------+--------+-----+
|   id|               title|votes_cnt|            rating|movie_id|genre|
+-----+--------------------+---------+------------------+--------+-----+
| 2422|Karate Kid, Part ...|       14|              1.75|    2422|Drama|
| 6793|    Beethoven (1992)|       11|1.7727272727272727|    6793|Drama|
| 2412|      Rocky V (1990)|       17|1.9411764705882353|    2412|Drama|
|46335|Fast and the Furi...|       11| 2.090909090909091|   46335|Drama|
| 1515|      Volcano (1997)|       15|               2.1|    1515|Drama|
| 4254|Crocodile Dundee ...|       13|2.1923076923076925|    4254|Drama|
|76175|Clash of the Tita...|       13|2.3076923076923075|   76175|Drama|
|  455|   Free Willy (1993)|       37| 2.391891891891892|     455|Drama|
|  519|    RoboCop 3 (1993)|       21|2.4047619047619047|     519|Drama|
|63992|     Twilight (2008)|       22| 2.409090909090909|   63992|Drama|
+-----+-------

<p><b>Задача 2.</b> Коллаборативная фильтрация <b><i>(5 баллов)</i></b></p>

<p><b style="color:green;">Вариант 1.</b> По схожести пользователей</p>
<p><b style="color:green;">Вариант 2.</b> По схожести объектов</p>

<p>6. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга</p>

In [9]:
train, test = votes.randomSplit([0.8, 0.2], seed=12)
train.persist()
test.persist()
print(train.count(), test.count())
# Разделил тестировачные данные на 4 партиции для параллельного подсчета
train.repartition(4)

80555 20281


DataFrame[user_id: bigint, movie_id: bigint, vote: double, date: bigint]

<p>7. Реализуйте коллаборативную фильтрацию в соотвествии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test</p>

In [10]:
#Ваш код

<p>8. Определите rmse для тестового подмножества</p>

In [11]:
#Ваш код

<p><b>Задача 3.</b> Факторизация матрицы <b><i>(3 балла)</i></b></p>

<p>9. Выберите модель ALS по минимальному значению rmse. Для этого используйте kfolds c k=4</p>

<div class="msg-block msg-warning">
  <p class="msg-text-warn">Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то rmse будет NaN. Поэтому заранее уберите из тестового/валидационного подмножества такие элементы</p>
</div>

In [12]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

# Параметры
rank = [5, 10, 15] # количество факторов
regParam = [0.001, 0.01, 0.1, 1, 10] # регуляризация

# Инициализация ALS.
# При прогнозировании с использованием оператора ALSModel, обычно встречаются пользователи или элементы в тестовом наборе данных, 
# которых не было во время обучения модели. По умолчанию Spark во время ALSModel.transform назначает NaN на прогнозы,
# когда пользователь или коэффициент элемента отсутствуют в модели.
# Параметр coldStartStrategy='drop' означает, что будет отбрасывать любые строки в DataFrame прогнозов, 
# которые содержат значения NaN.
als = ALS(coldStartStrategy='drop', userCol="user_id", itemCol="movie_id", ratingCol="vote")

# Инициализация RegressionEvaluator для выбора лучшей модели
evaluator = RegressionEvaluator(metricName="rmse", labelCol="vote", predictionCol="prediction")

# Инициализация ParamGridBuilder
param_grid = ParamGridBuilder()\
    .addGrid(als.rank, rank)\
    .addGrid(als.regParam, regParam)\
    .build()

# Инициализация CrossValidator
cv = CrossValidator(estimator=als, numFolds=4, estimatorParamMaps=param_grid, evaluator=evaluator)

print('Обучение модели с последующей выбором лучшей...')
models = cv.fit(train)

print('Метрики для каждой модели:')
print(models.avgMetrics)

Обучение модели с последующей выбором лучшей...
Метрики для каждой модели:
[1.2879213527273325, 1.0931058299459528, 0.9149025901813009, 1.3257023452278516, 3.6676806484297613, 1.438623797788002, 1.213718962142515, 0.9153725463471862, 1.3257040738208732, 3.6676806484297613, 1.576060667950038, 1.284156806047111, 0.9165412487091724, 1.3257044840171814, 3.6676806484297613]


In [15]:
best_model.rank

5

Сопаставляю метрики и параметры для каждой модели

In [47]:
list(zip(models.avgMetrics, param_grid))

[(1.2879213527273325,
  {Param(parent='ALS_e80e7351dd18', name='rank', doc='rank of the factorization'): 5,
   Param(parent='ALS_e80e7351dd18', name='regParam', doc='regularization parameter (>= 0).'): 0.001}),
 (1.0931058299459528,
  {Param(parent='ALS_e80e7351dd18', name='rank', doc='rank of the factorization'): 5,
   Param(parent='ALS_e80e7351dd18', name='regParam', doc='regularization parameter (>= 0).'): 0.01}),
 (0.9149025901813009,
  {Param(parent='ALS_e80e7351dd18', name='rank', doc='rank of the factorization'): 5,
   Param(parent='ALS_e80e7351dd18', name='regParam', doc='regularization parameter (>= 0).'): 0.1}),
 (1.3257023452278516,
  {Param(parent='ALS_e80e7351dd18', name='rank', doc='rank of the factorization'): 5,
   Param(parent='ALS_e80e7351dd18', name='regParam', doc='regularization parameter (>= 0).'): 1.0}),
 (3.6676806484297613,
  {Param(parent='ALS_e80e7351dd18', name='rank', doc='rank of the factorization'): 5,
   Param(parent='ALS_e80e7351dd18', name='regParam', 

<p>10. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов</p>

In [16]:
print('Предсказания на тестовых данных')
predictions = best_model.transform(test)
predictions.show()

Предсказания на тестовых данных
+-------+--------+----+----------+----------+
|user_id|movie_id|vote|      date|prediction|
+-------+--------+----+----------+----------+
|    133|     471| 4.0| 843491793|  2.764866|
|    182|     471| 4.5|1054779644| 3.4889863|
|    218|     471| 4.0|1111624874| 3.2175772|
|    474|     471| 3.0| 974668858|  3.839415|
|    387|     471| 3.0|1139047519| 2.8340988|
|    555|     471| 3.0| 978746933| 3.0487578|
|    520|     471| 5.0|1326609921| 3.5930803|
|    273|     471| 5.0| 835861348| 3.4615295|
|    104|     471| 4.5|1238111129| 3.6547296|
|    463|    1088| 3.5|1145460096| 3.2466214|
|    159|    1088| 4.0|1508641161| 3.3381646|
|     20|    1088| 4.5|1054147512| 3.5121129|
|    583|    1088| 3.5|1481474480|  3.347184|
|    555|    1088| 4.0| 978822670| 3.7007885|
|     84|    1088| 3.0| 860398568| 3.2135463|
|    268|    1238| 5.0| 940183153|  3.835485|
|     19|    1238| 3.0| 965705784| 3.6909757|
|    425|    1342| 3.5|1114173335| 2.4332187|
| 

In [17]:
print('RMSE: {}'.format(evaluator.evaluate(predictions)))
print('RMSE без NaN: {}'.format(evaluator.evaluate(predictions.na.drop())))

RMSE: 0.8822139125955473
RMSE без NaN: 0.8822139125955473


In [18]:
#Ваш код

<p><b>Задача 4.</b> Подготовка и отправка тетради с решением</p>

<p>11. Отправьте заполненную тетрадь c Вашим решением (адрес см. выше). В теме укажите: SD ДЗ3 группа Фамилия</p>

In [19]:
# Шаблон названия тетради (всё англ. буквами)
A№Задания_№Группа_Фамилия_ВариантЗадачи1_ВариантЗадачи2.ipynb

# Пример
A3_31_Ivanov_1_2.ipynb

SyntaxError: invalid character in identifier (<ipython-input-19-5ef9121810a6>, line 2)

<hr>
<div style="width:100%;"><p style="width:100%; text-align:center; font-weight:bold; font-size:18px;">FIN</p></div>